In [1]:
from ipynb.fs.defs.utils import read_lines 

In [2]:
puzzle_input = read_lines('day10.txt')

In [3]:
test_input = """..F7.
.FJ|.
SJ.L7
|F--J
LJ...""".splitlines()

In [4]:
def parse_map(schematics):
    map_dict = {}
    for y in range(len(schematics)):
        for x in range(len(schematics[y])):
            tile = schematics[y][x]
            if tile != '.':
                map_dict[(x, y)] = tile
    return map_dict

In [5]:
parse_map(test_input)

{(2, 0): 'F',
 (3, 0): '7',
 (1, 1): 'F',
 (2, 1): 'J',
 (3, 1): '|',
 (0, 2): 'S',
 (1, 2): 'J',
 (3, 2): 'L',
 (4, 2): '7',
 (0, 3): '|',
 (1, 3): 'F',
 (2, 3): '-',
 (3, 3): '-',
 (4, 3): 'J',
 (0, 4): 'L',
 (1, 4): 'J'}

In [6]:
def get_starting_point(pipe_map):
    for loc in pipe_map:
        if pipe_map[loc] == 'S':
            return loc

In [7]:
def get_connecting_pipes(loc, pipe_map):
    tile = pipe_map[loc]
    x, y = loc
    pipes = []
    if tile in ('|LJS'):
        pipes.append((x, y - 1))
    if tile in ('-LFS'):
        pipes.append((x + 1, y))
    if tile in ('|7FS'):
        pipes.append((x, y + 1))
    if tile in ('-J7S'):
        pipes.append((x - 1, y))
    pipes =  [p for p in pipes if pipe_map.get(p)]
    if tile == 'S':
        start_conns = []
        for p in pipes:
            if loc in get_connecting_pipes(p, pipe_map):
                start_conns.append(p)
        return start_conns
    else:
        return pipes

In [8]:
def get_loop(pipe_map):
    start = get_starting_point(pipe_map)
    path = [start]
    while path[-1] != start or len(path) < 2:
        connecting_pipes = get_connecting_pipes(path[-1], pipe_map)
        if len(path) < 2 or path[-2] == connecting_pipes[0]:
            path.append(connecting_pipes[1])
        else:
            path.append(connecting_pipes[0])
    return path

In [9]:
def part1(schematics):
    pipe_map = parse_map(schematics)
    path = get_loop(pipe_map)
    return len(path) // 2

In [10]:
part1(puzzle_input)

7063

In [11]:
def calc_shape(loc, pipe_map):
    connected_locs = get_connecting_pipes(loc, pipe_map)
    x, y = loc
    north = (x, y - 1) in connected_locs
    east = (x + 1, y) in connected_locs
    south = (x, y + 1) in connected_locs
    west = (x - 1, y) in connected_locs
    if north and south:
        return '|'
    if north and east:
        return 'L'
    if north and west:
        return 'J'
    if east and west:
        return '-'
    if east and south:
        return 'F'
    if south and west:
        return '7'
    print(f'Unknown shape at {loc}')

In [12]:
def part2(schematics, verbose = False):
    pipe_map = parse_map(schematics)
    path = get_loop(pipe_map)
    prev_edge_tile = None
    enclosing_count = 0
    for y in range(len(schematics)):
        inside = False
        for x in range(len(schematics[y])):
            loc = (x, y)
            tile = schematics[y][x]
            if tile == 'S':
                tile = calc_shape(loc, pipe_map)
            if loc in path:
                # check pipe divides space vertically: | or L--7 or F--J (arbitrary amount of - tiles)
                if tile == '|' or (prev_edge_tile == 'L' and tile == '7') or (prev_edge_tile == 'F' and tile == 'J'):
                    inside = not inside
                    prev_edge_tile = None
                    if verbose: print(inside, loc)
                if tile in 'LF':
                    prev_edge_tile = tile
                elif not tile == '-':
                    prev_edge_tile = None
            elif inside:
                prev_edge_tile = None
                enclosing_count += 1
                if verbose: print(f'inside: {loc}')
    return enclosing_count

In [13]:
part2(test_input, verbose=True)

True (2, 1)
False (3, 1)
True (1, 2)
inside: (2, 2)
False (4, 2)
True (0, 3)
False (4, 3)


1

In [14]:
test_input2 = """...........
.S-------7.
.|F-----7|.
.||OOOOO||.
.||OOOOO||.
.|L-7OF-J|.
.|II|O|II|.
.L--JOL--J.
.....O.....""".splitlines()

In [15]:
part2(test_input2, verbose=True)

True (1, 2)
False (9, 2)
True (1, 3)
False (2, 3)
True (8, 3)
False (9, 3)
True (1, 4)
False (2, 4)
True (8, 4)
False (9, 4)
True (1, 5)
False (4, 5)
True (8, 5)
False (9, 5)
True (1, 6)
inside: (2, 6)
inside: (3, 6)
False (4, 6)
True (6, 6)
inside: (7, 6)
inside: (8, 6)
False (9, 6)


4

In [16]:
part2(puzzle_input)

589